In [0]:
%pip install -U -q databricks-langchain langchain==0.3.7 faiss-cpu wikipedia langgraph==0.5.3  databricks_langchain

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

### Prompt Chaining - LangChain Expression Language.

In [0]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Databricks chat model wrapper
from databricks_langchain import ChatDatabricks  # pip install databricks-langchain


In [0]:
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-1-8b-instruct" # Model Serving endpoint name; other option see "Serving" under AI/ML tab (e.g. databricks-gpt-oss-20b)

llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.2)

In [0]:
# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
    "Extract the technical specifications from the following text:\n\n{text_input}"
)

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
    "Transform the following specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)

# --- Build the Chain using LCEL ---
extraction_chain = prompt_extract | llm | StrOutputParser()

full_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm
    | StrOutputParser()
)

# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)


--- Final JSON Output ---
Here is the JSON object based on the provided specifications:

```json
{
  "cpu": {
    "type": "Octa-core",
    "clock_speed": "3.5 GHz"
  },
  "memory": {
    "ram": "16GB"
  },
  "storage": {
    "type": "NVMe SSD",
    "capacity": "1TB"
  }
}
```

Note that I've used camelCase for the keys, which is a common convention in JSON. If you'd like to use underscores instead (e.g. `ram` becomes `ram`), I can modify the response accordingly. Let me know!


### LangChain Routing

In [0]:

# ---------- Simulated sub-agent handlers ----------
def booking_handler(request: str) -> str:
    print("\n--- DELEGATING TO BOOKING HANDLER ---")
    return f"Booking Handler processed request: '{request}'. Result: Simulated booking action."

def info_handler(request: str) -> str:
    print("\n--- DELEGATING TO INFO HANDLER ---")
    return f"Info Handler processed request: '{request}'. Result: Simulated information retrieval."

def unclear_handler(request: str) -> str:
    print("\n--- HANDLING UNCLEAR REQUEST ---")
    return f"Coordinator could not delegate request: '{request}'. Please clarify."

In [0]:
# ---------- Router chain ----------
coordinator_router_prompt = ChatPromptTemplate.from_messages([
    ("system", """Analyze the user's request and determine which specialist handler should process it.
- If the request is related to booking flights or hotels, output 'booker'.
- For all other general information questions, output 'info'.
- If the request is unclear or doesn't fit either category, output 'unclear'.
ONLY output one word: 'booker', 'info', or 'unclear'."""),
    ("user", "{request}")
])

coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()


In [0]:
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

# ---------- Delegation logic ----------
delegation_branch = RunnableBranch(
    (lambda x: x["decision"].strip() == "booker",
     RunnablePassthrough.assign(output=lambda x: booking_handler(x["input"]["request"]))),

    (lambda x: x["decision"].strip() == "info",
     RunnablePassthrough.assign(output=lambda x: info_handler(x["input"]["request"]))),

    # default
    RunnablePassthrough.assign(output=lambda x: unclear_handler(x["input"]["request"]))
)


In [0]:
coordinator_agent = (
    {"decision": coordinator_router_chain, "input": RunnablePassthrough()}
    | delegation_branch
    | (lambda x: x["output"])
)


In [0]:
print("--- Running with a booking request ---")
print(coordinator_agent.invoke({"request": "Book me a flight to London."}))

print("\n--- Running with an info request ---")
print(coordinator_agent.invoke({"request": "What is the capital of Italy?"}))

print("\n--- Running with an unclear request ---")
print(coordinator_agent.invoke({"request": "Tell me about quantum physics."}))


--- Running with a booking request ---

--- DELEGATING TO BOOKING HANDLER ---
Booking Handler processed request: 'Book me a flight to London.'. Result: Simulated booking action.

--- Running with an info request ---

--- DELEGATING TO INFO HANDLER ---
Info Handler processed request: 'What is the capital of Italy?'. Result: Simulated information retrieval.

--- Running with an unclear request ---

--- DELEGATING TO INFO HANDLER ---
Info Handler processed request: 'Tell me about quantum physics.'. Result: Simulated information retrieval.


### Parallelization


In [0]:
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough
import asyncio

# --- Define Independent Chains (run in parallel) ---
summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Summarize the following topic concisely:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

questions_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Generate three interesting questions about the following topic:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Identify 5-10 key terms from the following topic, separated by commas:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

In [0]:
# --- Build the Parallel + Synthesis Chain ---
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": questions_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # pass original input through
    }
)


In [0]:
synthesis_prompt = ChatPromptTemplate.from_messages([
    ("system", """Based on the following information:
Summary: {summary}
Related Questions: {questions}
Key Terms: {key_terms}
Synthesize a comprehensive answer."""),
    ("user", "Original topic: {topic}")
])

full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()


In [0]:
# --- Run the Chain ---
async def run_parallel_example(topic: str) -> None:
    if not llm:
        print("LLM not initialized. Cannot run example.")
        return

    print(f"\n--- Running Parallel LangChain Example for Topic: '{topic}' ---")
    try:
        response = await full_parallel_chain.ainvoke(topic)  # pass a single string
        print("\n--- Final Response ---")
        print(response)
    except Exception as e:
        print(f"\nAn error occurred during chain execution: {e}")

In [0]:
# Directly await the coroutine in notebook cell (Databricks supports top-level await)
test_topic = "The history of space exploration"
await run_parallel_example(test_topic)


--- Running Parallel LangChain Example for Topic: 'The history of space exploration' ---


/usr/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'run_parallel_example' was never awaited
  return compile(source, filename, mode, flags,



--- Final Response ---
The history of space exploration is a rich and fascinating story that spans over six decades, from the early years of the 1950s to the present day. It is a story of human ingenuity, technological advancements, and the unwavering desire to explore the unknown.

**Early Years (1950s-1960s)**

The early years of space exploration were marked by a series of significant milestones achieved by the Soviet Union and the United States. In 1957, the Soviet Union launched Sputnik 1, the first artificial satellite, into space, followed by the launch of Laika, the first living creature, into space in 1957. The United States responded by launching Explorer 1, the first successful American satellite, in 1958. The Soviet Union continued to push the boundaries of space exploration, sending Yuri Gagarin, the first human, into space in 1961, followed by Alan Shepard, the first American, in 1961.

**Moon Landings (1960s-1970s)**

The 1960s and 1970s saw the Soviet Union's Luna prog